In [ ]:
!pip install beautifulsoup4

In [205]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [206]:
def get_jobs_links(q):

    #faz a primeira pesquisa para captar o número de páginas resultantes da pesquisa
    response = requests.get(f'https://www.linkedin.com/jobs/search/?currentJobId=3640009990&f_TPR=r2592000&geoId=106057199&keywords={q}&location=Brasil&originalSubdomain=br', verify=False)
    soup = bs(response.text, 'html.parser')

    # recebe o número de resultados da pesquisa e cria uma lista com os parametros necessários para acessar todas as páginas.
    try:
        num_of_results = soup.find(class_ = 'results-context-header__job-count').text
        num_of_pages = int(int(num_of_results)/25)
        range_num_of_pages = list(range(0,num_of_pages))
        start_key_list = [x *25 for x in range_num_of_pages]
        start_key_list
    except ValueError:
        start_key_list = [0,25,50,75,100,125,150,175,200,225]
    
    links = {}
    #acessa cada uma das páginas do resultado da pesquisa
    for key in start_key_list:
        response = requests.get(f'https://www.linkedin.com/jobs/search/?currentJobId=3640009990&f_TPR=r2592000&geoId=106057199&keywords={q}&location=Brasil&originalSubdomain=br&start={key}', verify=False)
        soup = bs(response.text, 'html.parser')
        items = soup.findAll(class_='base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')
        
        
        #verifica se o título do item corresponde a pesquisa
        for item in items:
            job_title = str(item.span.text).strip()
            s = SequenceMatcher(None, q, job_title)
            if s.ratio() > 0.6:
                links[job_title] = item.get('href')
            
    return links 

    


In [207]:
q = 'engenheiro de dados'
list_of_links = get_jobs_links(q)


In [208]:
len(list_of_links)

28

In [209]:
"""list_of_prequisites = []
for link in list(list_of_links.values()):
    response = requests.get(link, verify=False)
    soup = bs(response.text, 'html.parser')
    try:
        list_of_prequisites.append(soup.find(class_='description__text description__text--rich').find_all('li'))
    except(AttributeError):
        pass
len(list_of_prequisites)"""

"list_of_prequisites = []\nfor link in list(list_of_links.values()):\n    response = requests.get(link, verify=False)\n    soup = bs(response.text, 'html.parser')\n    try:\n        list_of_prequisites.append(soup.find(class_='description__text description__text--rich').find_all('li'))\n    except(AttributeError):\n        pass\nlen(list_of_prequisites)"

In [210]:
list_of_prequisites = {}
for link in list(list_of_links.values()):
    response = requests.get(link, verify=False)
    soup = bs(response.text, 'html.parser')
    try:
        list_of_prequisites[link] = soup.find(class_='description__text description__text--rich').find_all('li')
    except(AttributeError):
        pass
len(list_of_prequisites)

22

In [211]:
#cria uma pasta para a query
#armazenar list_of_prequisites junto com timestamp (nosql)
#armazenar lista de links junto com timestamp (sql)
#recuperar lista de links e tranforma em texto apenas os links que não estiverem na lista
#append lista de links junto com timestamp
#importa todos os textos que estão marcados como a query especificada

In [212]:
"""dict_of_prequisites = {}
for link in list_of_links:
    response = requests.get(link, verify=False)
    soup = bs(response.text, 'html.parser')
    try:
        dict_of_prequisites[link] = (soup.find(class_='description__text description__text--rich').find_all('li'))
    except(AttributeError):
        pass
len(dict_of_prequisites)"""


"dict_of_prequisites = {}\nfor link in list_of_links:\n    response = requests.get(link, verify=False)\n    soup = bs(response.text, 'html.parser')\n    try:\n        dict_of_prequisites[link] = (soup.find(class_='description__text description__text--rich').find_all('li'))\n    except(AttributeError):\n        pass\nlen(dict_of_prequisites)"

In [213]:
text = str(list_of_prequisites.values()).replace('<li>','').replace('</li>','')\
    .replace('</li>',"")\
    .replace('<li>',"")\
    .replace('[',"")\
    .replace(']',"")\
    .replace('<br/>',"")\
    .replace('<br>',"")\
    .replace('</span>',"")\
    .replace('<span>',"")\
    .replace('.',"")\
    .replace(',',"")\
    .replace(';',"")\
    .lower()
    
words = str(list_of_prequisites.values()).replace('<li>','').replace('</li>','')\
    .replace('</li>',"")\
    .replace('<li>',"")\
    .replace('[',"")\
    .replace(']',"")\
    .replace('<br/>',"")\
    .replace('<br>',"")\
    .replace('</span>',"")\
    .replace('<span>',"")\
    .replace('.',"")\
    .replace(',',"")\
    .replace(';',"")\
    .lower()\
    .split()

stop_words = ['e','a', 'o', 'os', 'do', 'as', 'da', 'em', 'dos', 'se', 'no','com', 'na'\
        ,'para', 'ou', 'e a', 'é', '-', '/', 'de', 'um',' nos', 'que', 'das', 'por'\
        ,'como', '2', 'nos', 'off', 'e em', 'ao', 'com a', 'seu', 'são', 'etc', 'com o'\
        ,'esta', 'área', 'ano', 'dia', 'day','esta', 'nas', 'de', '–', 'à', 'x', 'ser', 'mas', 'sem', 'às', 'uma', ')', 'va', 'in', 'e/ou']
words = set(words)

for sw in stop_words:
    words.discard(sw)
words = list(words)

#a token is a "word" made of two or more words like "data scientist"
tokens = words
for w in range(len(words)):
    if w != len(words)-1:
        w1 = words[w]
        w2 = words[w+1]
        w1_w2 = w1 + ' ' + w2
        tokens.append(w1_w2)
        if w!= len(words)-2:
            w1 = words[w]
            w2 = words[w+1]
            w3 = words[w+2]
            w1_w2_w3 = w1 + ' ' + w2 + ' ' + w3
            tokens.append(w1_w2_w3)
        else:
            pass
    else:
        pass

set(tokens)

{'academia',
 'compras comunicação',
 'creche javascript)',
 'recuperabilidade',
 'negócio advisor totalpass',
 'planejamento hadoop',
 'emr',
 'tableau mês',
 'computação membros informação',
 'intelligence colaboradores',
 'envolvendo',
 'solução envolvendo',
 'relacionada',
 'extrair conhecimento',
 'impala) saúde mensalidade',
 'sabem',
 'familiaridade autodidata visão',
 'codificação preferência hive',
 'variável 13º conhecimentos',
 't-sql',
 '(r$12500000)',
 'híbridos',
 '(java',
 'base',
 'nosql dimensionais',
 'crescer melhorar',
 'ec2 arquiteturas ferramentas',
 'coparticipação',
 'captura normas',
 'resolução compromisso',
 '(plataforma elétrica',
 'iniciativas 30 componentes',
 'rest otimizadas',
 'time excelentes vivência',
 'melhorias liderar migração',
 'irá (cloudformation',
 'isso! outros',
 '#vemserbigdater',
 'experiência bibliotecas intermediário',
 'processamento incríveis paulo',
 'inovação',
 '(apache nosql) atuação',
 '(aws)',
 'software implementar',
 'airflow)

In [214]:
contagem_de_termos = []
for token in set(tokens):
    contagem_de_termos.append([token, text.count(token)])

In [215]:
df = pd.DataFrame(contagem_de_termos)
df[(df[1] >= 3)].sort_values(by=1, ascending=False).reset_index(drop=True).head(50)

,0,1
0,r,1104
1,c,741
2,l,539
3,dados,121
4,ti,97
5,bi,51
6,experiência,50
7,ir,43
8,conhecimento,38
9,data,31


In [ ]:

#adicionar o título de vaga
#adicionar o termo da pesquisa
#associar o link da publicação com a descrição
#retirar tags
#contar o número de ocorrências de cada palavra
#armazenar o resultado
#rodar o programa duas vezes por dia e adicionar apenas novos registros
#adicionar a data a criação da vaga
#adicionar a data da consulta